In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import sys

sys.path.append('./helpers_models/')
sys.path.append('./data_visualization_and_augmentations/')
sys.path.append('../torch_videovision/')
sys.path.append('./important_csvs/')

from helpers_resnet import *

resnet = torchvision.models.resnet50(pretrained=True)
adaptive_pooling = AdaptiveConcatPool2d()
head = Head()
resnet.avgpool = adaptive_pooling
resnet.fc = head

os.environ['CUDA_VISIBLE_DEVICES']='0,1,2'

resnet = resnet.cuda()

for param in resnet.parameters():
    param.requires_grad = False
    
for param in resnet.avgpool.parameters():
    param.requires_grad = True
    
for param in resnet.fc.parameters():
    param.requires_grad = True

resnet = nn.DataParallel(resnet)
check_freeze(resnet.module)

In [ ]:
from helpers_thresholds import *

In [ ]:
plt.rcParams['figure.figsize'] = (12,6)
font = {'family' : 'DejaVu Sans',  'weight' : 'normal',  'size'  : 40}
plt.rc('font', **font)

In [ ]:
tensor_transform = get_tensor_transform('ImageNet', False)
train_spat_transform = get_spatial_transform(2)
train_temp_transform = get_temporal_transform()
valid_spat_transform = get_spatial_transform(0)
valid_temp_transform = va.TemporalFit(size=16)

bs = 50
root_dir = '/media/scratch/astamoulakatos/nsea_video_jpegs/'
df = pd.read_csv('./small_dataset_csvs/events_with_number_of_frames_stratified.csv')
# df_train = get_df(df, 20, True, False, False)
# class_image_paths, end_idx, idx_label= get_indices(df_train, root_dir)
# train_loader = get_loader(1, 128, end_idx, class_image_paths, train_temp_transform, train_spat_transform, tensor_transform, False, True, True, 1)
df_valid = get_df(df, 20, False, True, False)
class_image_paths, end_idx, idx_label = get_indices(df_valid, root_dir)
valid_loader = get_loader(1, bs, end_idx, class_image_paths, valid_temp_transform, valid_spat_transform, tensor_transform, False, True, True, 1)
df_test = get_df(df, 20, False, False, True)
class_image_paths, end_idx, idx_label = get_indices(df_test, root_dir)
test_loader = get_loader(1, bs, end_idx, class_image_paths, valid_temp_transform, valid_spat_transform, tensor_transform, False, True, True, 1)

torch.cuda.empty_cache()

In [ ]:
load = True
if load:
    checkpoint = torch.load('/media/scratch/astamoulakatos/saved-resnet-models/fifth-small-aug-final/best-checkpoint-010epoch.pth')
    resnet.load_state_dict(checkpoint['model_state_dict'])
    print('loading pretrained freezed model!')

In [ ]:
device = torch.device('cuda')

In [ ]:
from predictions import *

In [ ]:
show_batch(valid_loader, 4)

In [ ]:
plot_predictions_actuals(valid_loader, 10, resnet, device)

In [ ]:
resnet.eval()
y_true = []
y_pred = []
y_512 = []
with torch.no_grad():
    for X, y in valid_loader:
        X = X.to(device)
        y = Variable(y.float()).to(device) 
        #X = X.permute(0,2,1,3,4)
        y = y.squeeze(dim=1)
        y = y.float()
        output5, output512 = resnet(X)
        y = y.detach().cpu()
        #loss = criterion(output, y)
        preds = torch.sigmoid(output5)
        preds = preds.to(torch.float32) 
        preds = preds.detach().cpu()
        y_pred.append(preds)
        y_true.append(y)
        y_512.append(output512.detach().cpu())

In [ ]:
len(y_pred), len(y_true), len(y_512)

In [ ]:
y_pr = torch.cat(y_pred)
y_tr = torch.cat(y_true)

In [ ]:
y_pr = y_pr.numpy()
y_tr = y_tr.numpy()

# Embeddings for Writer

In [ ]:
writer = SummaryWriter('runs/resnet2d_emb_3')

In [ ]:
import tensorflow as tf
import tensorboard as tb
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile

In [ ]:
y_pr.shape

In [ ]:
y_pr_tensor = torch.tensor(y_pr)

In [ ]:
y_512_tensor = torch.tensor(y_512_num)

In [ ]:
y_tr_list = y_tr.tolist()

In [ ]:
y_pr_tensor.shape

In [ ]:
writer.add_embedding(y_pr_tensor, metadata=y_tr_list)

In [ ]:
writer.add_embedding(y_512_tensor, metadata=y_tr_list)

In [ ]:
writer.close()

In [ ]:
y_512_tensor

In [ ]:
y_pr_tensor

In [ ]:
writer = SummaryWriter('runs/resnet2d_emb_4')

In [ ]:
writer.add_embedding(y_pr_tensor, metadata=y_tr_list)

In [ ]:
writer.close()

# Histograms of confidence scores

In [ ]:
y_pr.shape

In [ ]:
y_tr

In [ ]:
df_labels = pd.DataFrame(y_tr, columns=['exp','bur','fj','ande','fs'])

In [ ]:
df_labels['label'] = 'exp'

In [ ]:
df_labels

In [ ]:
for i in range(len(df_labels)):
    if df_labels.bur[i] == 1.0:
        df_labels.label[i] = 'bur'
    if df_labels.ande[i] == 1.0:
        df_labels.label[i] = 'anode'
    if df_labels.fj[i] == 1.0:
        df_labels.label[i] = 'fj'
    if df_labels.fs[i] == 1.0:
        df_labels.label[i] = 'fs'

In [ ]:
df_conf = pd.DataFrame(y_pr, columns=['exp_conf','bur_conf','fj_conf','ande_conf','fs_conf'])

In [ ]:
df_conf

In [ ]:
result = pd.concat([df_labels, df_conf], axis=1)

In [ ]:
result

In [ ]:
result.to_csv('./important_csvs/predictions_confidences_csvs/resnet2d-1frame.csv', index=False)

In [ ]:
result.ande_conf[result.ande==1].hist()
result.ande_conf[result.ande==0].hist()

In [ ]:
result.fj_conf[result.fj==1].hist()
result.fj_conf[result.fj==0].hist()

In [ ]:
result.exp_conf[result.exp==1].hist()
result.exp_conf[result.exp==0].hist()

In [ ]:
result.bur_conf[result.bur==1].hist()
result.bur_conf[result.bur==0].hist()

In [ ]:
result.fs_conf[result.fs==1].hist()
result.fs_conf[result.fs==0].hist()

In [ ]:
result

In [ ]:
result['exp_pred'] = 0.0
result['bur_pred'] = 0.0
result['fj_pred'] = 0.0
result['ande_pred'] = 0.0
result['fs_pred'] = 0.0

In [ ]:
for i in range(len(df_labels)):
    if result.bur_conf[i] >= 0.118:
        result.bur_pred= 1.0
    if result.ande_conf[i] >= 0.426:
        result.ande_pred[i] = 1.0
    if result.fj_conf[i] >= 0.179:
        result.fj_pred[i] = 1.0
    if result.fs_conf[i] >= 0.117:
        result.fs_pred[i] = 1.0
    if result.exp_conf[i] >= 0.778:
        result.exp_pred = 1.0

In [ ]:
result

In [ ]:
from reliability_diagrams import *